# TruLens Quickstart

In this quickstart you will create a RAG from scratch and learn how to log it and get feedback on an LLM response.

For evaluation, we will leverage the "hallucination triad" of groundedness, context relevance and answer relevance.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/truera/trulens/blob/main/trulens_eval/examples/quickstart/quickstart.ipynb)

In [ ]:
# ! pip install trulens_eval==0.19.2 chromadb==0.4.18 openai==1.3.7

In [1]:
import os, openai
from dotenv import load_dotenv
#os.environ["OPENAI_API_KEY"] = "..."
load_dotenv()

openai.api_key = os.getenv("OPENAI_API_KEY")
openai.api_base = os.getenv("OPENAI_API_BASE")
openai.api_type = os.getenv("OPENAI_API_TYPE")
openai.api_version = os.getenv("OPENAI_API_VERSION")
#print(openai.api_key)
#print(openai.api_base)
#print(openai.api_type)
#print(openai.api_version)

## Get Data

In this case, we'll just initialize some simple text in the notebook.

In [2]:
university_info = """
The University of Washington, founded in 1861 in Seattle, is a public research university
with over 45,000 students across three campuses in Seattle, Tacoma, and Bothell.
As the flagship institution of the six public universities in Washington state,
UW encompasses over 500 buildings and 20 million square feet of space,
including one of the largest library systems in the world.
"""

## Create Vector Store

Create a chromadb vector store in memory.

In [3]:
from openai import AzureOpenAI
oai_client = AzureOpenAI(
    api_key = os.getenv("OPENAI_API_KEY"),
    api_version = os.getenv("OPENAI_API_VERSION"),
    azure_endpoint = os.getenv("OPENAI_API_BASE"))

oai_client.embeddings.create(
        model="text-embedding-ada-002",
        input=university_info
    )

CreateEmbeddingResponse(data=[Embedding(embedding=[0.008559385314583778, -0.0036859966348856688, -0.005848066881299019, -0.02364625595510006, -0.020446009933948517, -0.008521287702023983, -0.00571789825335145, 0.003524079453200102, -0.0014326494419947267, -0.02264300361275673, -0.02801484428346157, 0.004771794192492962, 0.01780453883111477, -0.012508894316852093, -0.0033050149213522673, -0.01943005993962288, 0.011715183034539223, -0.03088490478694439, -0.007365643512457609, -0.017131470143795013, -0.026338525116443634, -0.014718587510287762, -0.00021311156160663813, -0.0046765487641096115, -0.011289753951132298, 0.022389017045497894, 0.0030589643865823746, -0.016128219664096832, 0.024662205949425697, -0.01040714606642723, -0.006825919728726149, -0.009842023253440857, -0.039495088160037994, 0.009892821311950684, -0.009086410515010357, 0.004873388912528753, -0.007422790862619877, 0.006359217222779989, 0.009251502342522144, -0.007365643512457609, 0.009861072525382042, 0.008178404532372952

In [4]:
import chromadb
from chromadb.utils.embedding_functions import OpenAIEmbeddingFunction

embedding_function = OpenAIEmbeddingFunction(api_key=os.environ.get('OPENAI_API_KEY'),
                                             api_base=os.environ.get('OPENAI_API_BASE'),
                                             api_type=os.environ.get('OPENAI_API_TYPE'),
                                             api_version=os.environ.get('OPENAI_API_VERSION'),
                                             model_name="text-embedding-ada-002")


chroma_client = chromadb.Client()
vector_store = chroma_client.get_or_create_collection(name="Universities",
                                                      embedding_function=embedding_function)

Add the university_info to the embedding database.

In [5]:
vector_store.add("uni_info", documents=university_info)

## Build RAG from scratch

Build a custom RAG from scratch, and add TruLens custom instrumentation.

In [6]:
from trulens_eval import Tru
from trulens_eval.tru_custom_app import instrument
tru = Tru()

🦑 Tru initialized with db url sqlite:///default.sqlite .
🛑 Secret keys may be written to the database. See the `database_redact_keys` option of `Tru` to prevent this.


In [7]:
class RAG_from_scratch:
    @instrument
    def retrieve(self, query: str) -> list:
        """
        Retrieve relevant text from vector store.
        """
        results = vector_store.query(
        query_texts=query,
        n_results=2
    )
        return results['documents'][0]

    @instrument
    def generate_completion(self, query: str, context_str: list) -> str:
        """
        Generate answer from context.
        """
        completion = oai_client.chat.completions.create(
        model="gpt-35-turbo",
        temperature=0,
        messages=
        [
            {"role": "user",
            "content":
            f"We have provided context information below. \n"
            f"---------------------\n"
            f"{context_str}"
            f"\n---------------------\n"
            f"Given this information, please answer the question: {query}"
            }
        ]
        ).choices[0].message.content
        return completion

    @instrument
    def query(self, query: str) -> str:
        context_str = self.retrieve(query)
        completion = self.generate_completion(query, context_str)
        return completion

rag = RAG_from_scratch()

## Set up feedback functions.

Here we'll use groundedness, answer relevance and context relevance to detect hallucination.

In [8]:
from trulens_eval import Feedback, Select
from trulens_eval.feedback import Groundedness
from trulens_eval.feedback.provider.openai import AzureOpenAI as fOpenAI

import numpy as np

# Initialize provider class
fopenai = fOpenAI(api_key=os.environ.get('OPENAI_API_KEY'),
                    azure_endpoint=os.environ.get('OPENAI_API_BASE'),
                    api_version=os.environ.get('OPENAI_API_VERSION'))

grounded = Groundedness(groundedness_provider=fopenai)

# Define a groundedness feedback function
f_groundedness = (
    Feedback(grounded.groundedness_measure_with_cot_reasons, name = "Groundedness")
    .on(Select.RecordCalls.retrieve.rets.collect())
    .on_output()
    .aggregate(grounded.grounded_statements_aggregator)
)

# Question/answer relevance between overall question and answer.
f_qa_relevance = (
    Feedback(fopenai.relevance_with_cot_reasons, name = "Answer Relevance")
    .on(Select.RecordCalls.retrieve.args.query)
    .on_output()
)

# Question/statement relevance between question and each context chunk.
f_context_relevance = (
    Feedback(fopenai.qs_relevance_with_cot_reasons, name = "Context Relevance")
    .on(Select.RecordCalls.retrieve.args.query)
    .on(Select.RecordCalls.retrieve.rets.collect())
    .aggregate(np.mean)
)

Arguments ['api_key', 'azure_endpoint', 'api_version'] are ignored as `client` was provided.


✅ In Groundedness, input source will be set to __record__.app.retrieve.rets.collect() .
✅ In Groundedness, input statement will be set to __record__.main_output or `Select.RecordOutput` .
✅ In Answer Relevance, input prompt will be set to __record__.app.retrieve.args.query .
✅ In Answer Relevance, input response will be set to __record__.main_output or `Select.RecordOutput` .
✅ In Context Relevance, input question will be set to __record__.app.retrieve.args.query .
✅ In Context Relevance, input statement will be set to __record__.app.retrieve.rets.collect() .


## Construct the app
Wrap the custom RAG with TruCustomApp, add list of feedbacks for eval

In [9]:
from trulens_eval import TruCustomApp
tru_rag = TruCustomApp(rag,
    app_id = 'RAG v1',
    feedbacks = [f_groundedness, f_qa_relevance, f_context_relevance])

## Run the app
Use `tru_rag` as a context manager for the custom RAG-from-scratch app.

In [10]:
with tru_rag as recording:
    rag.query("When was the University of Washington founded?")
    rag.query("How many buildings are there in University of Washington?")
    rag.query("When did University of Washington become public?")
    rag.query("When did University of Washington become private?")
    rag.query("How many total students has had the Washington University in its life?")

Number of requested results 2 is greater than number of elements in index 1, updating n_results = 1
Number of requested results 2 is greater than number of elements in index 1, updating n_results = 1
Number of requested results 2 is greater than number of elements in index 1, updating n_results = 1
Number of requested results 2 is greater than number of elements in index 1, updating n_results = 1


In [11]:
tru.get_leaderboard(app_ids=["RAG v1"])

,Answer Relevance,Groundedness,Context Relevance,latency,total_cost
app_id,,,,,
RAG v1,1.0,1.0,1.0,1.0,0.000211


In [15]:
tru.run_dashboard()

Starting dashboard ...
Config file already exists. Skipping writing process.
Credentials file already exists. Skipping writing process.


Accordion(children=(VBox(children=(VBox(children=(Label(value='STDOUT'), Output())), VBox(children=(Label(valu…